In [7]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from scipy.integrate import odeint
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)
# 设置中文字体为 Mac 系统中文字体
plt.rcParams["font.sans-serif"] = ["Arial Unicode MS"]
plt.rcParams["axes.unicode_minus"] = False

## 1. 数据预处理

In [8]:
## 1. 数据预处理
# 读取网络数据
edge_file = "data/raw/european/powergridEU_E.csv"
coord_file = "data/raw/european/powergridEU_V.csv"

# 读取数据
edges = pd.read_csv(edge_file, header=None).values.tolist()
coords = pd.read_csv(coord_file, header=None)  # 节点坐标数据

# 创建无向图
g = ig.Graph.TupleList(edges, directed=False)
g.vs["name"] = [int(v) for v in g.vs["name"]]  # 节点名转为整数

In [9]:
# 创建节点ID到索引的映射以加速查找
name_to_index = {v["name"]: idx for idx, v in enumerate(g.vs)}
# 批量添加坐标数据
latitudes = [None] * g.vcount()
longitudes = [None] * g.vcount()
for _, row in coords.iterrows():
    node_id = int(row[0])
    if node_id in name_to_index:  # 防止数据不匹配导致的错误
        idx = name_to_index[node_id]
        latitudes[idx] = row[1]
        longitudes[idx] = row[2]

# 批量设置顶点属性
g.vs["latitude"] = latitudes
g.vs["longitude"] = longitudes

# 检查数据完整性
assert all(lat is not None for lat in latitudes), "存在缺失的纬度数据"
assert all(lon is not None for lon in longitudes), "存在缺失的经度数据"

In [10]:
# 向量化的 Haversine 距离计算
def haversine_distances(lats, lons):
    """
    计算所有节点对之间的 Haversine 距离（km）
    使用向量化操作提高效率
    """
    # 转换为弧度
    lats_rad = np.radians(lats)
    lons_rad = np.radians(lons)
    # 使用广播计算差值
    lat_matrix = lats_rad[:, np.newaxis] - lats_rad[np.newaxis, :]
    lon_matrix = lons_rad[:, np.newaxis] - lons_rad[np.newaxis, :]
    # Haversine 公式
    a = np.sin(lat_matrix / 2) ** 2 + np.cos(lats_rad[:, np.newaxis]) * np.cos(lats_rad[np.newaxis, :]) * np.sin(
        lon_matrix / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))  # 添加 clip 避免数值误差

    # 地球半径 (km)
    R = 6371
    return R * c

# 计算节点间地理距离矩阵（用于动态传播因子）
distances = haversine_distances(np.array(latitudes), np.array(longitudes))

In [12]:
from itertools import combinations

from scipy.sparse import coo_matrix, csr_matrix, lil_matrix

# 一阶邻接矩阵 A（稀疏格式）
edges = np.array(g.get_edgelist())
n_nodes, n_edges = g.vcount(), len(edges)
adj_sparse = coo_matrix((np.ones(len(edges)), (edges[:, 0], edges[:, 1])),
                    shape=(g.vcount(), g.vcount())).tocsr()
edge_dict = {tuple(sorted(e)): idx for idx, e in enumerate(edges)}

In [13]:
# 构建 B1
B1 = lil_matrix((n_nodes, n_edges)) # lil_matrix 更适合动态插入
for idx, (i, j) in enumerate(edges):
    B1[i, idx] = 1  # 起点+1
    # B1[j, idx] = -1  # 终点-1
    B1[j, idx] = 1  # 无向边：两个节点均标记为+1
B1 = B1.tocsr()  # 转换为CSR用于后续计算

In [14]:
# 检测所有3节点环路
triangles = []
for node in g.vs:
    neighbors = node.neighbors()
    for pair in combinations(neighbors, 2):
        if g.are_adjacent(pair[0], pair[1]):
            triangle = sorted([node.index, pair[0].index, pair[1].index])
            triangles.append(tuple(triangle))
triangles = list(set(triangles))  # 去重

In [15]:
# 构建 B2（统一顺时针定向）
B2 = lil_matrix((n_edges, len(triangles)))
for t_idx, triangle in enumerate(triangles):
    # 定义顺时针边顺序：A→B, B→C, C→A
    ordered_edges = [
        tuple(sorted((triangle[0], triangle[1]))),
        tuple(sorted((triangle[1], triangle[2]))),
        tuple(sorted((triangle[2], triangle[0])))
    ]
    for i, edge in enumerate(ordered_edges):
        if edge in edge_dict:
            B2[edge_dict[edge], t_idx] = 1 if i == 0 else -1
B2 = B2.tocsr()

In [16]:
# 计算 Hodge Laplacian
L1 = B1.T @ B1 + B2 @ B2.T

# 正确归一化（处理空矩阵）
B2_squared = B2 @ B2.T
if B2_squared.ndim == 0:
    D1_diag = np.zeros(n_edges)  # 无三角形时度数为0
else:
    # 代码中通过 B2_squared.sum(axis=1) 计算 D1_diag，这实际是每行元素的总和 ，而非每个边的度数（即边参与的三角形数量）
    # D1_diag = np.array(B2_squared.sum(axis=1)).flatten()  # 显式转换为密集数组
    D1_diag = B2_squared.diagonal()  # 修正后的代码：提取对角线元素
    D1_diag = np.maximum(D1_diag, 0)  # 强制非负，避免负值
# 避免除零错误（添加1e-6偏移量）
D1_diag_safe = D1_diag + 1e-6
D1_inv_sqrt = csr_matrix(np.diag(1.0 / np.sqrt(D1_diag_safe)))  # 计算平方根倒数
L1_tilde = D1_inv_sqrt @ L1 @ D1_inv_sqrt

In [17]:
print(B1.shape, B2.shape, L1.shape, L1_tilde.shape)

(13478, 16922) (16922, 1213) (16922, 16922) (16922, 16922)


In [18]:
print(f"检测到 {len(triangles)} 个三角形环路")
print(f"B2 shape: {B2.shape}")
print(f"B2_squared shape: {B2_squared.shape}")

检测到 1213 个三角形环路
B2 shape: (16922, 1213)
B2_squared shape: (16922, 16922)


In [24]:
class ClimateParams:
    flood_max_lat = 55      # 洪水风险最大纬度
    flood_scale = 10        # 洪水风险衰减系数
    heat_min_lat = 40       # 热浪风险最小纬度
    heat_scale = 15         # 热浪风险增长系数
    drought_offset = 10     # 干旱经度偏移量
    drought_scale = 20      # 干旱风险缩放系数

latitudes = np.array(g.vs["latitude"], dtype=np.float32)
longitudes = np.array(g.vs["longitude"], dtype=np.float32)
# 洪水风险：纬度越低风险越高
F = np.maximum(0, (ClimateParams.flood_max_lat - latitudes) / ClimateParams.flood_scale)
# 热浪风险：纬度越高风险越高
H = np.maximum(0, (latitudes - ClimateParams.heat_min_lat) / ClimateParams.heat_scale)
# 干旱风险：经度越东风险越高
D = np.maximum(0, (longitudes + ClimateParams.drought_offset) / ClimateParams.drought_scale)
"""
# 归一化风险值到 [0, 1] 范围
scaler = MinMaxScaler()
F = scaler.fit_transform(F.reshape(-1, 1)).flatten()
H = scaler.fit_transform(H.reshape(-1, 1)).flatten()
D = scaler.fit_transform(D.reshape(-1, 1)).flatten()
"""
X_feature = np.column_stack([latitudes, longitudes, F, H, D])
print(f"气候特征生成完成，形状：{X_feature.shape}")

气候特征生成完成，形状：(13478, 5)


In [47]:
edges = np.array(g.get_edgelist())  # 边列表 (16922×2)
print(edges.shape)
edge_features = []
for idx, (i, j) in enumerate(edges):
    # 节点特征拼接
    node_features = np.concatenate([X_feature[i], X_feature[j]])
    # 边的距离（仅相邻节点）
    distance = distances[i, j]
    # 合并为边特征
    edge_feature = np.hstack([node_features, distance])
    edge_features.append(edge_feature)
edge_features = np.array(edge_features, dtype=np.float32)
print(f"边特征生成完成，形状：{edge_features.shape}（应为 16922×11）")

(16922, 2)
边特征生成完成，形状：(16922, 11)（应为 16922×11）


##  2. 高阶图神经网络

In [25]:
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import relu, sigmoid

class HoSC(nn.Module):
    """HodgeSimplexConv"""
    def __init__(self, input_dim, hidden_dim, output_dim, L1_tilde):
        super(HoSC, self).__init__()
        self.L1_tilde = L1_tilde  # 归一化Hodge Laplacian (稀疏矩阵)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, Z):
        # 将边特征转换为PyTorch稀疏张量
        Z = torch.FloatTensor(Z).unsqueeze(-1)  # (n_edges, input_dim, 1)

        # 第一层单纯形卷积
        Z1 = self.L1_tilde @ Z  # 使用稀疏矩阵乘法
        Z1 = self.fc1(Z1.squeeze(-1))
        Z1 = relu(Z1)

        # 第二层单纯形卷积
        Z2 = self.L1_tilde @ Z1.unsqueeze(-1)
        Z2 = self.fc2(Z2.squeeze(-1))

        return Z2  # 输出边级嵌入 (n_edges, output_dim)


class NodeFailurePredictor(nn.Module):
    def __init__(self, hoc_model, B1_abs):
        super(NodeFailurePredictor, self).__init__()
        self.hoc_model = hoc_model
        self.B1_abs = B1_abs  # 边界矩阵的绝对值（稀疏矩阵）

    def forward(self, Z_edge):
        # 计算节点失效概率
        edge_embeddings = self.hoc_model(Z_edge)  # (n_edges, output_dim)
        node_logits = self.B1_abs.T @ edge_embeddings  # (n_nodes, output_dim)
        node_prob = sigmoid(node_logits.sum(axis=1))  # 聚合边特征并压缩到[0,1]
        return node_prob

In [41]:
def generate_edge_features(disaster_type, distances, edges, F, H, D):
    """根据灾害类型生成边特征"""
    n_edges = len(edges)
    edge_features = []

    if disaster_type == "flood":
        risk_coupling = F
    elif disaster_type == "heatwave":
        risk_coupling = H
    elif disaster_type == "drought":
        risk_coupling = D
    else:
        raise ValueError("Unsupported disaster type")

    # 构建边特征矩阵
    for u, v in edges:
        distance = distances[u, v]
        risk_sum = risk_coupling[u] + risk_coupling[v]
        edge_features.append([distance, risk_sum])

    return np.array(edge_features, dtype=np.float32)

In [47]:
# 超参数设置
input_dim = 2  # 地理距离 + 灾害风险
hidden_dim = 16
output_dim = 8
learning_rate = 0.01
weight_decay = 1e-4
epochs = 100

In [48]:
# 预处理：将B1转换为绝对值稀疏矩阵
B1_abs = abs(B1).tocsr()

# 初始化模型
hoc_model = HoSC(input_dim, hidden_dim, output_dim, L1_tilde)
predictor = NodeFailurePredictor(hoc_model, abs(B1).tocsr())
optimizer = optim.Adam(predictor.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

In [50]:
# 生成训练数据（以洪水为例）
def train(disaster_type):
    edge_features = generate_edge_features(disaster_type, distances, edges, F, H, D)

    # 创建标签（假设纬度<45的节点中10%失效）
    labels = np.zeros(len(g.vs))
    southern_nodes = np.where(latitudes < 45)[0]
    failed_nodes = np.random.choice(southern_nodes, size=int(0.1*len(southern_nodes)), replace=False)
    labels[failed_nodes] = 1
    labels = torch.FloatTensor(labels)

    for epoch in range(epochs):
        optimizer.zero_grad()
        prob = predictor(torch.FloatTensor(edge_features))
        loss = criterion(prob, labels)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")


# 运行训练
train("flood") # 或 "heatwave", "drought"

ValueError: could not interpret dimensions